### Inicio

In [1]:
from datetime import date
import os
os.environ["AWS_PROFILE"] = "men"
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [ ]:
#!pip install langchain
#!pip install langchain_aws

   ---------------------------------------- 0.0/95.4 kB ? eta -:--:--
   ------------ --------------------------- 30.7/95.4 kB 640.0 kB/s eta 0:00:01
   ---------------------------------------- 95.4/95.4 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/139.2 kB ? eta -:--:--
   ---------------------------------------- 139.2/139.2 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   --- ------------------------------------ 1.0/13.3 MB 32.4 MB/s eta 0:00:01
   ----- ---------------------------------- 1.9/13.3 MB 29.4 MB/s eta 0:00:01
   -------- ------------------------------- 2.9/13.3 MB 23.3 MB/s eta 0:00:01
   ----------------- ---------------------- 5.9/13.3 MB 34.0 MB/s eta 0:00:01
   ---------------------------- ----------- 9.5/13.3 MB 43.4 MB/s eta 0:00:01
   ------------------------------------ --- 12.1/13.3 MB 50.4 MB/s eta 0:00:01
   ---------------------------------------  13.3/13.3 MB 73.1 MB/s eta 0:00:01
   

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.7.0 requires botocore<1.31.65,>=1.31.16, but you have botocore 1.36.2 which is incompatible.


In [5]:
from langchain.prompts import PromptTemplate

In [ ]:
from langchain_aws import BedrockLLM
from langchain_aws import ChatBedrock, ChatBedrockConverse

In [21]:
params = {
    "class_name":"pedo",
    "grade":"italiano"
}

In [22]:
def get_relevant_info():
    prompt_content = """
    Como se dice "{class_name}" en ingles y en {grade}?
    """
    
    return PromptTemplate(
        input_variables=["class_name", "grade"],
        template=prompt_content,
    )


In [23]:
from langchain_core.output_parsers import StrOutputParser

In [24]:
llm = ChatBedrockConverse(model="anthropic.claude-3-sonnet-20240229-v1:0", 
                          temperature=0.02, 
                          max_tokens=None)

relevance = get_relevant_info() | llm | StrOutputParser()s

relevance_params = {
    "class_name": params["class_name"],
    "grade": params["grade"]
}

relevante = relevance.invoke(relevance_params)

In [25]:
print(relevante)

En inglés, la palabra "pedo" se traduce como "fart". En italiano, se dice "peto".

Sin embargo, es importante tener en cuenta que estas palabras pueden considerarse groseras o inapropiadas en ciertos contextos formales. Se recomienda utilizar un lenguaje más educado o eufemismos en situaciones que lo requieran.


In [26]:
def get_wimbledon_final():
    prompt_content = """
    Quien fue el ganador de wimbledon en el año {year}?
    """
    
    return PromptTemplate(
        input_variables=["year"],
        template=prompt_content,
    )

In [27]:
get_wimbledon_final = get_wimbledon_final() | llm | StrOutputParser()

print(get_wimbledon_final.invoke({"year": "2024"}))

Lamentablemente, no tengo información sobre los ganadores de Wimbledon en el año 2024, ya que ese torneo aún no se ha disputado. Wimbledon es uno de los torneos de tenis más importantes del mundo y se juega anualmente en Londres, Inglaterra. Los ganadores de las ediciones más recientes han sido:

2022:
- Individuales masculino: Novak Djokovic (Serbia)
- Individuales femenino: Elena Rybakina (Kazajistán)

2021: 
- Individuales masculino: Novak Djokovic (Serbia)  
- Individuales femenino: Ashleigh Barty (Australia)

2020: Torneo cancelado debido a la pandemia de COVID-19

2019:
- Individuales masculino: Novak Djokovic (Serbia)
- Individuales femenino: Simona Halep (Rumanía)

La edición de 2024 se disputará presumiblemente entre finales de junio y mediados de julio de ese año en las canchas de césped de el All England Lawn Tennis and Croquet Club. Tendremos que esperar hasta entonces para conocer a los nuevos campeones.
